In [1]:
# from datasets import load_dataset
# dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train", cache_dir="./data")

In [2]:
# print(dataset)
# print(dataset[0:5])
# print(dataset[0:5]["text"])

In [3]:
# def get_training_corpus():
#     for i in range(0, len(dataset), 1000):
#         yield dataset[i : i + 1000]["text"]

In [4]:
# with open("./data/wikitext-2.txt", "w", encoding="utf-8") as f:
#     for i in range(len(dataset)):
#         f.write(dataset[i]["text"] + "\n")

In [5]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

# tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer = Tokenizer(models.WordLevel(unk_token="[UNK]"))

In [6]:
# tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [7]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [8]:
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))
normalized = tokenizer.normalizer.normalize_str("123 4567 89012")   # string
print(normalized)

hello how are u?
123 4567 89012


In [9]:
# tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
# tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

In [10]:
# pre_tokenized = tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")
# print(pre_tokenized)
# pre_tokenized = tokenizer.pre_tokenizer.pre_tokenize_str("1 23 456 7890")
# print(pre_tokenized)

In [11]:
# pre_tokenizer = pre_tokenizers.WhitespaceSplit()
# pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

In [12]:
# pre_tokenizer = pre_tokenizers.Sequence(
#     [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
# )
# pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

In [13]:
import data_helpers
from config import config

In [14]:
df = data_helpers.read_large_excel(config['data'], "contest_total_data")
df = data_helpers.improve_contest_data(df)
print(df.columns)
teams = list(df['teams_home_team_id']) + list(df['teams_away_team_id'])
print(len(teams), teams)
teams_string = [str(team) for team in teams]
teams_text = " ".join(teams_string)
print(teams_text)

f = open("./team_ids_text.txt", "w", encoding="utf-8")
f.write(teams_text)
f.close()

Index(['fixture_id', 'teams_home_team_id', 'teams_away_team_id',
       'fixture_date', 'fixture_venue', 'league_id', 'league_name',
       'league_country', 'league_season', 'league_round',
       ...
       'teams_away_all_last_matches_goalsAvg_goalsAgainst_away',
       'teams_away_last_h2h_played_home', 'teams_away_last_h2h_played_away',
       'teams_away_last_h2h_wins_home', 'teams_away_last_h2h_wins_away',
       'teams_away_last_h2h_draws_home', 'teams_away_last_h2h_draws_away',
       'teams_away_last_h2h_loses_home', 'teams_away_last_h2h_loses_away',
       'outcome'],
      dtype='object', length=105)
94800 [407, 7412, 2312, 12711, 563, 406, 9486, 473, 8550, 9565, 8617, 2079, 9938, 451, 252, 5176, 7320, 352, 699, 1122, 7792, 2774, 7294, 3114, 1484, 12928, 617, 845, 5949, 146, 1405, 2592, 106, 7028, 6615, 5248, 8662, 8531, 1828, 9446, 4195, 5757, 2753, 2960, 9510, 1945, 4687, 4275, 3112, 356, 8350, 3858, 1149, 1846, 1456, 6671, 9576, 3604, 8523, 4325, 9535, 3121, 2932, 2540, 

In [15]:
# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
# trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

special_tokens = ["[UNK]", "[HOME]", "[AWAY]"]
trainer = trainers.WordLevelTrainer(vocab_size=len(teams_string)+3, special_tokens=special_tokens)

In [16]:
# tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [17]:
# # tokenizer.model = models.WordPiece(unk_token="[UNK]")
# tokenizer.model = models.WordLevel(unk_token="[UNK]")
# tokenizer.train(["./data/wikitext-2.txt"], trainer=trainer)

In [18]:
tokenizer.train(["./team_ids_text.txt"], trainer=trainer)

In [19]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[UNK]', '[UNK]', '[UNK]', '[UNK]']


In [20]:
encoding = tokenizer.encode("407 7412 7320 400 123456789")
print(encoding.tokens)

['407', '7412', '7320', '400', '[UNK]']


In [21]:
ukn_token_id = tokenizer.token_to_id("[UNK]")
home_token_id = tokenizer.token_to_id("[HOME]")
away_token_id = tokenizer.token_to_id("[AWAY]")
print(ukn_token_id, home_token_id, away_token_id)

0 1 2


In [22]:
# tokenizer.post_processor = processors.TemplateProcessing(
#     single=f"[CLS]:0 $A:0 [SEP]:0",
#     pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
#     special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
# )

In [23]:
# encoding = tokenizer.encode("Let's test this tokenizer.")
# print(encoding.tokens)

In [24]:
encoding = tokenizer.encode("1 2 3 what? 333 -5 4)e_t grw0")
print(encoding.tokens)
print(encoding.type_ids)
print(encoding.ids)

['1', '2', '3', '[UNK]', '333', '[UNK]', '[UNK]', '[UNK]']
[0, 0, 0, 0, 0, 0, 0, 0]
[6866, 6385, 4456, 0, 1434, 0, 0, 0]


In [25]:
# encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
# print(encoding.tokens)
# print(encoding.type_ids)

In [26]:
# tokenizer.decoder = decoders.WordPiece(prefix="##")
tokenizer.decoder = decoders.WordPiece(prefix=" ")

In [27]:
tokenizer.decode(encoding.ids)

'1 2 3 333'

In [28]:
tokenizer.save("./data/tokenizer.json")

In [29]:
new_tokenizer = Tokenizer.from_file("./data/tokenizer.json")

In [30]:
# from transformers import PreTrainedTokenizerFast

# wrapped_tokenizer = PreTrainedTokenizerFast(
#     tokenizer_object=tokenizer,
#     # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
#     unk_token="[UNK]",
#     pad_token="[PAD]",
#     cls_token="[CLS]",
#     sep_token="[SEP]",
#     mask_token="[MASK]",
# )

In [31]:
# from transformers import BertTokenizerFast

# wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [32]:
[item for item in dir(new_tokenizer) if not item.startswith('_')]

['add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'encode_special_tokens',
 'from_buffer',
 'from_file',
 'from_pretrained',
 'from_str',
 'get_added_tokens_decoder',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']

In [33]:
# print(new_tokenizer.vocab.shape, new_tokenizer.get_vocab_size(), new_tokenizer.get_vocab_path(), new_tokenizer.get_reserved_tokens())
# print(tokeniznew_tokenizerer.vocab[:30], new_tokenizer.vocab[-30:], new_tokenizer.vocab[2000:2010])

In [34]:
# print(new_tokenizer.vocab)
# print(len(new_tokenizer.vocab))

In [35]:
# print(wrapped_tokenizer.vocab.shape, wrapped_tokenizer.get_vocab_size(), wrapped_tokenizer.get_vocab_path(), wrapped_tokenizer.get_reserved_tokens())
# print(wrapped_tokenizer.vocab[:30], wrapped_tokenizer.vocab[-30:], wrapped_tokenizer.vocab[2000:2010])